# Projet crash test - analyse résultat élections présidentielles 1er tour

## Transformation du df pour pouvoir analyser

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as mtick

pres_df = pd.read_csv("prez_tour1_clean.csv", delimiter=";")
pres_df

,id_circo,Code du département,Libellé du département,Code de la circonscription,Libellé de la circonscription,Inscrits,Abstentions,% Abs/Ins,Votants,% Vot/Ins,...,Voix.10,% Voix/Ins.10,% Voix/Exp.10,N°Panneau.11,Sexe.11,Nom.11,Prénom.11,Voix.11,% Voix/Ins.11,% Voix/Exp.11
0,0,1,Ain,1,1ère circonscription,85723,18592,"21,69",67131,"78,31",...,437,"0,51","0,67",12,M,DUPONT-AIGNAN,Nicolas,1753,"2,04","2,67"
1,1,1,Ain,2,2ème circonscription,99383,19917,"20,04",79466,"79,96",...,467,"0,47","0,6",12,M,DUPONT-AIGNAN,Nicolas,1883,"1,89","2,42"
2,2,1,Ain,3,3ème circonscription,81500,20440,"25,08",61060,"74,92",...,424,"0,52","0,71",12,M,DUPONT-AIGNAN,Nicolas,1820,"2,23","3,04"
3,3,1,Ain,4,4ème circonscription,94359,19826,"21,01",74533,"78,99",...,445,"0,47","0,61",12,M,DUPONT-AIGNAN,Nicolas,2020,"2,14","2,78"
4,4,1,Ain,5,5ème circonscription,77144,18766,"24,33",58378,"75,67",...,399,"0,52","0,7",12,M,DUPONT-AIGNAN,Nicolas,1522,"1,97","2,67"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,572,ZZ,Français établis hors de France,7,7ème circonscription,122158,69559,"56,94",52599,"43,06",...,349,"0,29","0,67",12,M,DUPONT-AIGNAN,Nicolas,639,"0,52","1,23"
573,573,ZZ,Français établis hors de France,8,8ème circonscription,130065,106455,"81,85",23610,"18,15",...,133,"0,1","0,57",12,M,DUPONT-AIGNAN,Nicolas,300,"0,23","1,28"
574,574,ZZ,Français établis hors de France,9,9ème circonscription,121122,81344,"67,16",39778,"32,84",...,135,"0,11","0,34",12,M,DUPONT-AIGNAN,Nicolas,381,"0,31","0,97"
575,575,ZZ,Français établis hors de France,10,10ème circonscription,103910,63575,"61,18",40335,"38,82",...,163,"0,16","0,41",12,M,DUPONT-AIGNAN,Nicolas,530,"0,51","1,33"


In [3]:
nuance_df = pd.read_csv("nuances_partis_candidats.csv", delimiter=",",encoding="latin-1")
nuance_df

,Sigle,Nuance,Etiquette,Candidat,Parti
0,DXG,Divers extrême gauche,Gauche,Philippe POUTOU,NPA
1,DXG,Divers extrême gauche,Gauche,Nathalie ARTHAUD,LO
2,NUP,Nouvelle union populaire écologique et sociale,Gauche,Jean-Luc MÉLENCHON,LFI
3,NUP,Nouvelle union populaire écologique et sociale,Gauche,Fabien ROUSSEL,PCF
4,NUP,Nouvelle union populaire écologique et sociale,Gauche,Yannick JADOT,EELV
5,NUP,Nouvelle union populaire écologique et sociale,Gauche,Anne HIDALGO,PS
6,RDG,Parti radical de gauche,Gauche,NaN,NaN
7,DVG,Divers gauche,Gauche,NaN,NaN
8,ECO,Ecologistes,Gauche,NaN,NaN
9,DIV,Divers,Gauche,NaN,NaN


In [4]:
#Transformer le dataframe pour qu'il y ait 12 lignes par circonscription, une pour chaque candidat
df = pres_df

# On ne garde que la première partie du tableau
df=df.loc[:, ['id_circo', 'Code du département', 'Code de la circonscription', 'Libellé du département', 'Libellé de la circonscription','Sexe', 'Nom', 'Prénom', 'Voix', '% Voix/Ins', '% Voix/Exp']]
df_final=df

# On sélectionne chaque partie du df initial correspondant à un candidat, et on l'ajoute au df initial
for i in range(1,12):
  df = pres_df.loc[:, ['id_circo', 'Code du département', 'Libellé du département', 'Libellé de la circonscription',f'Sexe.{i}', f'Nom.{i}', f'Prénom.{i}', f'Voix.{i}', f'% Voix/Ins.{i}', f'% Voix/Exp.{i}']]
  df = df.rename(columns={f'Sexe.{i}':'Sexe', f'Nom.{i}':'Nom', f'Prénom.{i}':'Prénom', f'Voix.{i}':'Voix', f'% Voix/Ins.{i}':'% Voix/Ins', f'% Voix/Exp.{i}':'% Voix/Exp'})
  df_final = pd.concat([df_final,df], ignore_index=True)

In [5]:
#Modification du df pour faire jointure
df = df_final
# Utilisation de apply() avec une fonction lambda pour créer la colonne "Candidat"
df['Candidat'] = df.apply(lambda row: row['Prénom'] + ' ' + row['Nom'], axis=1, )
# Réarrangement des colonnes pour placer "Candidat" en 5ème position
columns = df.columns.tolist()
columns = columns[:4] + ['Candidat'] + columns[4:10]
df = df.reindex(columns=columns)
#suppression du prénom et nom
df.pop('Prénom')
df.pop('Nom')
df_candidat = df

In [6]:
#jointure
df = df_candidat

df = pd.merge(df_candidat,nuance_df, how="left", on="Candidat")

df_cand_circo_nuance = df 
df_cand_circo_nuance


,id_circo,Code du département,Code de la circonscription,Libellé du département,Candidat,Libellé de la circonscription,Sexe,Voix,% Voix/Ins,Sigle,Nuance,Etiquette,Parti
0,0,1,1.0,Ain,Nathalie ARTHAUD,1ère circonscription,F,317,"0,37",DXG,Divers extrême gauche,Gauche,LO
1,1,1,2.0,Ain,Nathalie ARTHAUD,2ème circonscription,F,354,"0,36",DXG,Divers extrême gauche,Gauche,LO
2,2,1,3.0,Ain,Nathalie ARTHAUD,3ème circonscription,F,275,"0,34",DXG,Divers extrême gauche,Gauche,LO
3,3,1,4.0,Ain,Nathalie ARTHAUD,4ème circonscription,F,376,"0,4",DXG,Divers extrême gauche,Gauche,LO
4,4,1,5.0,Ain,Nathalie ARTHAUD,5ème circonscription,F,336,"0,44",DXG,Divers extrême gauche,Gauche,LO
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6919,572,ZZ,NaN,Français établis hors de France,Nicolas DUPONT-AIGNAN,7ème circonscription,M,639,"0,52",DVD,Divers droite,Droite,Debout la France
6920,573,ZZ,NaN,Français établis hors de France,Nicolas DUPONT-AIGNAN,8ème circonscription,M,300,"0,23",DVD,Divers droite,Droite,Debout la France
6921,574,ZZ,NaN,Français établis hors de France,Nicolas DUPONT-AIGNAN,9ème circonscription,M,381,"0,31",DVD,Divers droite,Droite,Debout la France
6922,575,ZZ,NaN,Français établis hors de France,Nicolas DUPONT-AIGNAN,10ème circonscription,M,530,"0,51",DVD,Divers droite,Droite,Debout la France


In [7]:
df['Code de la circonscription'] = df['Code de la circonscription'].astype(int)

IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer

In [ ]:
#export
df_cand_circo_nuance.to_csv('candidat_1t_pres_circo_nuance.csv', index=False)